# PART 2
# Analysis of data obtained from Bioscreen experiment - data frames after pyphe-growthcurves pipeline; Data visualization (Heatmaps)


The notebook is a second part of script for analysis the data obtained from Bioscreen or any other multi-well plate experiment. On previous notebook we prepared the data frames with results of the maximum slope and lag phase, according to script described by Stephan Kamrad frim Bahler's Lab.

The data frames should be available in folders:

                ./Data/Bioscreen_data_YES/
                ./Data/Bioscreen_data_EMM/

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob

%matplotlib inline

# ncPAP deletion mutants cultivated in YES supplemented with stressors

### Import data

In [3]:
# YES:
path_YES =r'./Data/Bioscreen_data_YES/'
filenames_YES = glob.glob(path_YES + "*.csv")

In [ ]:
# Import data for YES
# We have only one file with results of ncPAP deletion mutants cultivated in YES medium supplemented with stressors

YES_1 = pd.read_csv(filenames_YES[0])

Now we have new data frames YES_1 and EMM_1-EMM_5.

# YES medium
## Preprocessing of results

In [ ]:
# We have only one file with results of ncPAP deletion mutants cultivated in YES supplemented with stressors
# DataFrame: YES_1

# Add new column 'Medium':
YES_1['Medium'] = 'YES'
# Make temporarily a new data frame containing the split value of 'Mutant' column:
new = YES_1["Mutants"].str.split("_", n = 1, expand = True) 
# Add column 'Yeast' from 'new' data frame:
YES_1["Yeast"]= new[0] 
# Delete the rows containing 'Δlsm': 
YES_1 = YES_1[~YES_1.Mutants.str.contains('Δlsm')] 
# Add column 'Stress' from 'new' data frame: 
YES_1["Stress"]= new[1]
# Add column '_Medium_Stress':
YES_1['Medium_Stress'] = YES_1[['Medium','Stress']].apply(lambda x: '_'.join(x[x.notnull()]), axis = 1)
# Save your preprocessed Data Frame:
YES_1.to_csv("DataFrames/YES_1_preprocessed.csv")

# Check the content of YES_1:
#YES_1.head()

# YES medium
## Make the pivot tables with maximum slope and lag phase

In [ ]:
# Make the pivot tables with maximum slope and lag phase
YES_pivot_lagphase = YES_1.pivot_table(index='Yeast', columns='Medium_Stress', values='lag')
YES_pivot_maxslope = YES_1.pivot_table(index='Yeast', columns='Medium_Stress', values='max_slope')


# Save the pivot tables as csv files: 
YES_pivot_lagphase.to_csv('DataFrames/YES_pivot_lagphase.csv')
YES_pivot_maxslope.to_csv('DataFrames/YES_pivot_maxslope.csv')


In [ ]:
# Print the pivot tables (if you want):
print("results of lag phase:")
print(YES_pivot_lagphase)
print("")
print("")
print("results of Maximum slope:")
print(YES_pivot_maxslope)

## Normalization of laghase and maximum slope

In [ ]:
# Data for each mutant were normalized to growth on media without the stressor (EMM or YES, respectively).

# Lag phase
YES_lagphase_divirder = YES_pivot_lagphase['YES']
YES_lagphase_divirder = YES_lagphase_divirder.transpose()
YES_lagphase_after_transpose = YES_pivot_lagphase.transpose()

YES_pivot_lagphase_normalized = YES_lagphase_after_transpose/YES_lagphase_divirder
YES_pivot_lagphase_normalized = YES_pivot_lagphase_normalized.transpose()


# Maximum slope
YES_maxslope_divirder = YES_pivot_maxslope['YES']
YES_maxslope_divirder = YES_maxslope_divirder.transpose()
YES_maxslope_after_transpose = YES_pivot_maxslope.transpose()

YES_pivot_maxslope_normalized = YES_maxslope_after_transpose/YES_maxslope_divirder
YES_pivot_maxslope_normalized = YES_pivot_maxslope_normalized.transpose()

# Save the pivot tables as csv files: 
YES_pivot_lagphase_normalized.to_csv('DataFrames/YES_pivot_lagphase_normalized.csv')
YES_pivot_maxslope_normalized.to_csv('DataFrames/YES_pivot_maxslope_normalized.csv')

In [ ]:
# Print the pivot tables with normalized results (if you want):
print("Normalized results of lag phase:")
print(YES_pivot_lagphase_normalized)
print("")
print("")
print("Normalized results of Maximum slope:")
print(YES_pivot_maxslope_normalized)

## YES - Heatmaps preparation

### Dobór wielkości czcionki na osiach x i y

In [ ]:
# Helper funtion with interactive widgets to select proper size of labels and title

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact
def heat_map_fun(label_size=15,title_size=15):
    def heat_map_elegant(title,path_to_save,df,cmap='RdYlBu',ylabel='Fission yeast name'):
        ax =sns.heatmap(df,cmap=cmap,linewidths=5)
        #with sns.axes_style("white"):
        #    ax = sns.heatmap(df, vmax=.3, square=True)
        fig=ax.figure #wyciagamy dostep do figure
        fig.set_size_inches(10,8)
        ax.set(title=title,
               ylabel=ylabel,
              xlabel='Growth medium/ stress conditions',
              )
        ax.xaxis.label.set_size(label_size)
        ax.yaxis.label.set_size(label_size)
        ax.title.set_size(title_size)
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")
        plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
                 rotation_mode="anchor")

        plt.setp(ax.get_yticklabels(), style="italic")    
        #fig.savefig(path_to_save+".pdf",dpi=150)
        #fig.savefig(path_to_save+".png",dpi=150)
        return ax,fig
    return heat_map_elegant("The normalized maximum slope of fission yeast in YES",
                            "Plots_Heatmaps//YES_max_slope_RdYlBu_znormalizowana",YES_pivot_maxslope_normalized)

## Colors of heatmap
We used two kinds of colors of cmap:
1. 'RdYlBu' for lag phase
2. 'RdYlBu_r' ('RdYlBu' with reversed colors) for maximum slope

You can find other colors for label here: https://matplotlib.org/examples/color/colormaps_reference.html


In [ ]:
# This function reverses the colors of chosen cmap and makes new cmap named "oldname_r":

reversed('RdYlBu')

#Output:
#'RdYlBu_r'

## Make the heatmaps

In [ ]:
def heat_map_elegant_YES(title,path_to_save,df,cmap,ylabel='Name of ncPAP deletion mutants'):
    '''
    This function will create a heatmap and save it as pdf and png.
    
    title: title of plot [string]
    path_to_save: path to directory where the heatmap will be saved [string]
    df: 
    
    '''
    ax =sns.heatmap(df,cmap=cmap,linewidths=1)
    #with sns.axes_style("white"):
    #    ax = sns.heatmap(df, vmax=.3, square=True)
    fig=ax.figure #wyciagamy dostep do figure
    fig.set_size_inches(6,9)
    ax.set(title=title,
           ylabel=ylabel,
          xlabel='YES & stress conditions')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    ax.title.set_size(20)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor", fontsize=20)
    plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
             rotation_mode="anchor", fontsize=20)
    
    plt.setp(ax.get_yticklabels(), style="italic")   
    plt.rcParams.update({'legend.labelspacing':0.25})
    fig.tight_layout()

    fig.savefig(path_to_save+".pdf",dpi=150)
    fig.savefig(path_to_save+".png",dpi=150)
    return ax,fig

In [ ]:
# Heatmap of the normalized maximum slope of fission yeast in YES

heat_map_elegant_YES("The normalized maximum slope of fission yeast in YES",
                     "Plots_Heatmaps//YES_max_slope_RdYlBu_znormalizowana",
                     YES_pivot_maxslope_normalized,'RdYlBu_r')

In [ ]:
# Heatmap of the normalized lag phase of fission yeast in YES

heat_map_elegant_YES("The normalized lag phase of fission yeast in YES",
                     "Plots_Heatmaps//YES_lagphase_RdYlBu_znormalizowana",
                     YES_pivot_lagphase_normalized,'RdYlBu')

# EMM medium
## Preprocessing of results

In [8]:
# Make a path for data and list of files:
path_EMM =r'./Data/Bioscreen_data_EMM/'
filenames_EMM = glob.glob(path_EMM + "*.csv")
filenames_EMM_short = filenames_EMM


In [20]:
import os.path
old_names=[os.path.splitext(os.path.basename(name))[0] for name in filenames_EMM]

In [9]:
EMM_list=[pd.read_csv(filename_EMM) for filename_EMM in filenames_EMM ]

In [ ]:
list_of_dataframes_EMM = (EMM_1, EMM_2, EMM_3, EMM_4, EMM_5)

In [10]:
# We have 5 filse with results of ncPAP deletion mutants cultivated in YES supplemented with stressors
# DataFrames: EMM_1 - EMM_5

# Please do it only once - the second one will change the output



def put_yeast_stress(EMM):
    EMM['Medium'] = 'EMM'        #Add new column 'Medium'    
    new_i = EMM["Mutants"].str.split("_", n = 1, expand = True)  # Make temporarily 'new' data frame    
    EMM["Yeast"]= new_i[0] # Add column 'Yeast' from 'new' data frame    
    EMM["Stress"]= new_i[1]  # Add column 'Stress' from 'new' data frame
#    i = i[i['Yeast']!='Δlsm'] # Delete the rows containing 'Δlsm'       
    EMM['Medium_Stress'] = EMM[['Medium','Stress']].apply(lambda x: '_'.join(x[x.notnull()]),
                                                      axis = 1) # Add column '_Medium_Stress'  
    
    

In [11]:
for EMM in EMM_list:
    put_yeast_stress(EMM)

In [22]:
for old_name,EMM in zip(old_names,EMM_list): # takes a pair one from first one from second list
    print(os.path.join("DataFrames","preprocessed_%s.csv" % old_name))
    EMM.to_csv(os.path.join("DataFrames","preprocessed_%s.csv" % old_name))

DataFrames/preprocessed_EMM_20190530.csv
DataFrames/preprocessed_EMM_20190606.csv
DataFrames/preprocessed_EMM_20190624.csv
DataFrames/preprocessed_EMM_20190613.csv
DataFrames/preprocessed_EMM_20190612.csv


In [23]:
lagphase_pivot_EMM_list=[EMM.pivot_table(index='Yeast', columns='Medium_Stress', values='lag')
                        for EMM in EMM_list]

maxslope_pivot_EMM_list=[EMM.pivot_table(index='Yeast', columns='Medium_Stress', values='max_slope')
                        for EMM in EMM_list]

lagph

In [24]:
for old_name,EMM in zip(old_names, lagphase_pivot_EMM_list):
    print(os.path.join("DataFrames","lagphase_pivot_%s.csv" % old_name))
    EMM.to_csv(os.path.join("DataFrames","lagphase_pivot_%s.csv" % old_name))

DataFrames/lagphase_pivot_EMM_20190530.csv
DataFrames/lagphase_pivot_EMM_20190606.csv
DataFrames/lagphase_pivot_EMM_20190624.csv
DataFrames/lagphase_pivot_EMM_20190613.csv
DataFrames/lagphase_pivot_EMM_20190612.csv


In [25]:
for old_name,EMM in zip(old_names, maxslope_pivot_EMM_list):
    print(os.path.join("DataFrames","maxslope_pivot_%s.csv" % old_name))
    EMM.to_csv(os.path.join("DataFrames","maxslope_pivot_%s.csv" % old_name))

DataFrames/maxslope_pivot_EMM_20190530.csv
DataFrames/maxslope_pivot_EMM_20190606.csv
DataFrames/maxslope_pivot_EMM_20190624.csv
DataFrames/maxslope_pivot_EMM_20190613.csv
DataFrames/maxslope_pivot_EMM_20190612.csv


In [ ]:
list_of_laghase_pivot_EMM = (lagphase_pivot_EMM_1,lagphase_pivot_EMM_2,lagphase_pivot_EMM_3,
                             lagphase_pivot_EMM_4,lagphase_pivot_EMM_5)

list_of_maxslope_pivot_EMM = (maxslope_pivot_EMM_1,lagphase_pivot_EMM_2,lagphase_pivot_EMM_3,
                             maxslope_pivot_EMM_4,lagphase_pivot_EMM_5)

In [ ]:
names_of_laghase_pivot_EMM = {"lagphase_pivot_EMM_1": lagphase_pivot_EMM_1,
                              "lagphase_pivot_EMM_2":lagphase_pivot_EMM_2,
                              "lagphase_pivot_EMM_3":lagphase_pivot_EMM_3,
                             "lagphase_pivot_EMM_4":lagphase_pivot_EMM_4,
                              "lagphase_pivot_EMM_5":lagphase_pivot_EMM_5}

names_of_maxslope_pivot_EMM = {"maxslope_pivot_EMM_1": maxslope_pivot_EMM_1,
                              "maxslope_pivot_EMM_2":maxslope_pivot_EMM_2,
                              "maxslope_pivot_EMM_3":maxslope_pivot_EMM_3,
                             "maxslope_pivot_EMM_4":maxslope_pivot_EMM_4,
                              "maxslope_pivot_EMM_5":maxslope_pivot_EMM_5}

In [ ]:
list_of_laghase_pivot_EMM[1]

In [ ]:
# Data for each mutant were normalized to growth on media without the stressor (EMM or YES, respectively).

# Lag phase
for i in list_of_laghase_pivot_EMM:
    for number in range (1,5):
        lagphase_divider_EMM = i['EMM']
        lagphase_divider_EMM[number] = lagphase_divider_EMM[number].transpose()
    
        lagphase_after_transpose_EMM[number] = i.transpose()
        pivot_lagphase_normalized_EMM[number] = lagphase_after_transpose_EMM[number]/lagphase_divider_EMM[number]
        pivot_lagphase_normalized_EMM[number] = pivot_lagphase_normalized_EMM[number].transpose()
   

In [ ]:
 pivot_lagphase_normalized_EMM_2
    

In [ ]:
# Data for each mutant were normalized to growth on media without the stressor (EMM or YES, respectively).

# Lag phase

lagphase_divider_EMM_1 = lagphase_pivot_EMM_1['EMM']
lagphase_divider_EMM_1 = lagphase_divider_EMM_1.transpose()
lagphase_after_transpose_EMM_1 = lagphase_pivot_EMM_1.transpose()
pivot_lagphase_normalized_EMM_1 = lagphase_after_transpose_EMM_1/lagphase_divider_EMM_1
pivot_lagphase_normalized_EMM_1 = pivot_lagphase_normalized_EMM_1.transpose()
pivot_lagphase_normalized_EMM_1

# Maximum slope
#YES_maxslope_divider = YES_pivot_maxslope['YES']
#YES_maxslope_divider = YES_maxslope_divrder.transpose()
#YES_maxslope_after_transpose = YES_pivot_maxslope.transpose()

#YES_pivot_maxslope_normalized = YES_maxslope_after_transpose/YES_maxslope_divirder
#YES_pivot_maxslope_normalized = YES_pivot_maxslope_normalized.transpose()

# Save the pivot tables as csv files: 
#YES_pivot_lagphase_normalized.to_csv('DataFrames/YES_pivot_lagphase_normalized.csv')
#YES_pivot_maxslope_normalized.to_csv('DataFrames/YES_pivot_maxslope_normalized.csv')

In [ ]:
pivot_lagphase_normalized_EMM_1

In [ ]:
# normalizacja laghase i maxslope - dzielę wyniki dla stresów przez wyniki dla samej pożywki

# Lagfaza
dzielnik_lagphase = lag_pivot_EMM['EMM']
dzielnik_lagphase = dzielnik_lagphase.transpose()

Lag_po_transpozycji = lag_pivot_EMM.transpose()

normalized_lagphase_EMM_pivot = Lag_po_transpozycji/dzielnik_lagphase




# maxslope
dzielnik_maxslope = maxslope_pivot_EMM['EMM']
dzielnik_maxslope = dzielnik_maxslope.transpose()

maxslope_po_transpozycji1 = maxslope_pivot_EMM.transpose()

normalized_maxslope_EMM_pivot = maxslope_po_transpozycji1/dzielnik_maxslope


# zapisuje wyniki jako obiekty dla konkternej daty w formacie DDMMRRRR
normalized_lagphase_EMM_pivot_24062019 = normalized_lagphase_EMM_pivot
normalized_maxslope_EMM_pivot_24062019 = normalized_maxslope_EMM_pivot

# mam przygotowane pivot tables dla YES - lagphase and maxslope osobno

# Tutaj podglądam moje nowe znowrmalizowane pivot tables:
print("Normalized results of lag phase:")
print(normalized_lagphase_EMM_pivot)
print("")
print("")
print("Normalized results of Maximum slope:")
print(normalized_maxslope_EMM_pivot)

### Join pivot tables (lag phase and maxslope separately) after all of EMM experiments together

In [ ]:
#LAG PHASE

frames_lagphase_EMM = [normalized_lagphase_EMM_pivot_30052019,
                       normalized_lagphase_EMM_pivot_06062019, normalized_lagphase_EMM_pivot_12062019, 
                       normalized_lagphase_EMM_pivot_13062019,normalized_lagphase_EMM_pivot_24062019]

joined_frames_lagphase_EMM = pd.concat(frames_lagphase_EMM)

In [ ]:
#LAG PHASE

frames_maxslope_EMM = [normalized_maxslope_EMM_pivot_30052019,
                       normalized_maxslope_EMM_pivot_06062019, normalized_maxslope_EMM_pivot_12062019, 
                       normalized_maxslope_EMM_pivot_13062019,normalized_maxslope_EMM_pivot_24062019]

joined_frames_maxslope_EMM = pd.concat(frames_maxslope_EMM )
joined_frames_maxslope_EMM


In [ ]:
#imporp do csv

joined_frames_maxslope_EMM.to_csv('joined_frames_maxslope_EMM.csv')
joined_frames_lagphase_EMM.to_csv('joined_frames_lagphase_EMM.csv')

# w plikach ręcznie :( wywalam dislsm i lsm oraz powtórki EMM

# Mogę puścić kod od tego momentu - mam wygenerowany plik csv

In [ ]:
results_EMM_all_lag = pd.read_csv('joined_frames_lagphase_EMM.csv')
results_EMM_all_maxslope = pd.read_csv('joined_frames_maxslope_EMM.csv')


results_EMM_all_maxslope = results_EMM_all_maxslope.set_index('Medium_Stress')
results_EMM_all_lag = results_EMM_all_lag.set_index('Medium_Stress')

In [ ]:
results_EMM_all_maxslope = results_EMM_all_maxslope.transpose()
results_EMM_all_lag = results_EMM_all_lag.transpose()

In [ ]:
results_EMM_all_lag

In [ ]:
results_EMM_all_maxslope

In [ ]:
maxslope_pivot_YES

## Tutaj mam poprawnie zrobione heatmapy EMM - do czyszczenia

#### ładuję pliki 
    results_EMM_all_lag
    results_EMM_all_maxslope

In [ ]:
results_EMM_all_lag

In [ ]:
#heatmap maxslope EMM


df = results_EMM_all_maxslope
fig, ax = plt.subplots(figsize=a4_dims)
plt.fontsize=20
#plt.xlim([0,8])
#plt.ylim([0.01,150])
plt.xticks(fontsize=18, rotation=90)
plt.yticks(fontsize=18)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
#plt.legend(loc=2, prop={'size': 6})
plt.title("The maximum slope of the growth curve of ncPAP deletion mutants in EMM medium", fontsize=20)
plt.xlabel("Medium & stress conditions",fontsize=20)
plt.ylabel("Deletion mutants",fontsize=20)
#plt =sns.heatmap(df,cmap=plt.cm.RdYlBu)
#with sns.axes_style("white"):
#    ax = sns.heatmap(df, vmax=.3, square=True)

plt.savefig("EMM_max_slope_RdYlBu_znormalizowana.pdf")
plt.savefig("EMM_max_slope_RdYlBu_znormalizowana.png")

In [ ]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact
def heat_map_fun(label_size=15,title_size=15):
    def heat_map_elegant(title,path_to_save,df,cmap='RdYlBu',ylabel='Fission yeast name'):
        ax =sns.heatmap(df,cmap=cmap,linewidths=5)
        #with sns.axes_style("white"):
        #    ax = sns.heatmap(df, vmax=.3, square=True)
        fig=ax.figure #wyciagamy dostep do figure
        fig.set_size_inches(10,8)
        ax.set(title=title,
               ylabel=ylabel,
              xlabel='Growth medium/ stress conditions',
              )
        ax.xaxis.label.set_size(label_size)
        ax.yaxis.label.set_size(label_size)
        ax.title.set_size(title_size)
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")
        plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
                 rotation_mode="anchor")

        plt.setp(ax.get_yticklabels(), style="italic")    
        #fig.savefig(path_to_save+".pdf",dpi=150)
        #fig.savefig(path_to_save+".png",dpi=150)
        return ax,fig
    return heat_map_elegant("EMM_max_slope_RdYlBu_znormalizowana","ploty/EMM_max_slope_RdYlBu_znormalizowana",results_EMM_all_lag)

In [ ]:
def heat_map_elegant(title,path_to_save,df,cmap='RdYlBu_r',ylabel='Name of ncPAP deletion mutants'):
    ax =sns.heatmap(df,cmap=cmap,linewidths=1)
    #with sns.axes_style("white"):
    #    ax = sns.heatmap(df, vmax=.3, square=True)
    fig=ax.figure #wyciagamy dostep do figure
    fig.set_size_inches(5,5.9)
    ax.set(title=title,
           ylabel=ylabel,
          xlabel='EMM & stress conditions')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    ax.title.set_size(20)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor", fontsize=20)
    plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
             rotation_mode="anchor", fontsize=20)
    
    plt.setp(ax.get_yticklabels(), style="italic")   
    plt.rcParams.update({'legend.labelspacing':0.25})
    fig.tight_layout()

    fig.savefig(path_to_save+".pdf",dpi=150)
    fig.savefig(path_to_save+".png",dpi=150)
    return ax,fig

#ax,fig=heat_map_elegant("EMM_max_slope_RdYlBu_znormalizowana","ploty/EMM_max_slope_RdYlBu_znormalizowana",df);

In [ ]:
ax,fig=heat_map_elegant("The normalized maximum slope of fission yeast in EMM medium",
                        "ploty/EMM_maxslope_LL",results_EMM_all_maxslope);

In [ ]:
# odwraca kolory w wybramej cmap i tworzy nowy cmap o nazwie "staranazwa_r"
reversed('RdYlBu')

In [ ]:
def heat_map_elegant_YES(title,path_to_save,df,cmap='RdYlBu_r',ylabel='Fission yeast name'):
    ax =sns.heatmap(df,cmap=cmap,linewidths=1)
    #with sns.axes_style("white"):
    #    ax = sns.heatmap(df, vmax=.3, square=True)
    fig=ax.figure #wyciagamy dostep do figure
    fig.set_size_inches(5.5,9)
    ax.set(title=title,
           ylabel=ylabel,
          xlabel='YES & stress conditions')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    ax.title.set_size(20)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor", fontsize=20)
    plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
             rotation_mode="anchor", fontsize=20)
    
    plt.setp(ax.get_yticklabels(), style="italic")   
    plt.rcParams.update({'legend.labelspacing':0.25})
    fig.tight_layout()

    fig.savefig(path_to_save+".pdf",dpi=150)
    fig.savefig(path_to_save+".png",dpi=150)
    return ax,fig

In [ ]:
ax,fig=heat_map_elegant("The normalized Lag phase of fission yeast in EMM medium",
                        "ploty/EMM_lagphase_LL",df_yes_maxslope);

In [ ]:
def heat_map_elegant_YES(title,path_to_save,df,cmap='RdYlBu_r',ylabel='Fission yeast name'):
    ax =sns.heatmap(df,cmap=cmap,linewidths=1)
    #with sns.axes_style("white"):
    #    ax = sns.heatmap(df, vmax=.3, square=True)
    fig=ax.figure #wyciagamy dostep do figure
    fig.set_size_inches(5.5,9)
    ax.set(title=title,
           ylabel=ylabel,
          xlabel='YES & stress conditions')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    ax.title.set_size(20)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor", fontsize=20)
    plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
             rotation_mode="anchor", fontsize=20)
    
    plt.setp(ax.get_yticklabels(), style="italic")   
    plt.rcParams.update({'legend.labelspacing':0.25})
    fig.tight_layout()

    fig.savefig(path_to_save+".pdf",dpi=150)
    fig.savefig(path_to_save+".png",dpi=150)
    return ax,fig

#ax,fig=heat_map_elegant("EMM_max_slope_RdYlBu_znormalizowana","ploty/EMM_max_slope_RdYlBu_znormalizowana",df)

In [ ]:
#YES
df_yes_lagphase = normalized_lagphase_YES_pivot.transpose()
df_yes_maxslope = normalized_maxslope_YES_pivot.transpose()


In [ ]:
ax,fig=heat_map_elegant_YES("The normalized maximum slope of fission yeast in YES medium",
                        "ploty/YES_maxslope_LL",df);

In [ ]:
#YES

ax,fig=heat_map_elegant("The normalized maximum slope \n of fission yeast in YES medium",
                        "ploty/YES_maxslope_LL",normalized_maxslope_YES_pivot)

In [ ]:
ax,fig=heat_map_elegant_YES("The normalized Lag phase \n of fission yeast in YES medium",
                        "ploty/YES_lagphase_LL",df_yes_lagphase);

In [ ]:
ax.get_label

In [ ]:
ax.figure.get_children()

In [ ]:
fig.artists

In [ ]:
sns.heatmap(results_EMM_all_maxslope, cmap='RdYlBu')

In [ ]:
results_EMM_all_maxslope

In [ ]:
sns.heatmap(results_EMM_all_lag, cmap='RdYlBu')

# Nieudane próby modyfikacji heatmapy

In [ ]:
#heatmap lagphase

fig, ax = plt.subplots()
#im = ax.imshow(harvest)
ax.fontsize=20
#plt.xlim([0,8])
#plt.ylim([0.01,150])
plt.xticks(fontsize=18, rotation=90)
plt.yticks(fontsize=18)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
#plt.legend(loc=2, prop={'size': 6})
plt.title("Survival of ncPAP deletion mutants in EMM medium", fontsize=20)
plt.xlabel("Stress conditions",fontsize=20)
plt.ylabel("Deletion mutants",fontsize=20)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

plt =sns.heatmap(results_EMM_all_lag, cmap='Spectral')
#with sns.axes_style("white"):
#    ax = sns.heatmap(df, vmax=.3, square=True)
plt.tight_layout()
plt.savefig("EMM_lag_Spectral_znormalizowana.pdf")
plt.savefig("EMM_lag_Spectral_znormalizowana.png")